In [1]:
pip install python-vlc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show python-vlc

Name: python-vlc
Version: 3.0.18122
Summary: VLC bindings for python.
Home-page: http://wiki.videolan.org/PythonBinding
Author: Olivier Aubert
Author-email: contact@olivieraubert.net
License: LGPL-2.1+
Location: /Users/kim-sung-eun/anaconda3/lib/python3.10/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
#! /usr/bin/env python3
#
# PyQt5-based video-sync example for VLC Python bindings
# Copyright (C) 2009-2010 the VideoLAN team
#
# This program is free software; you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation; either version 2 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program; if not, write to the Free Software
# Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston MA 02110-1301, USA.
#
"""
This module contains a bare-bones VLC player class to play videos.

Author: Saveliy Yusufov, Columbia University, sy2685@columbia.edu
Date: 25 January 2019
"""

import os
import sys
import queue
import platform

from PyQt5 import QtWidgets, QtGui, QtCore
import vlc

from __future__ import unicode_literals
import yt_dlp as youtube_dl

class MiniPlayer(QtWidgets.QMainWindow):
    """Stripped-down PyQt5-based media player class to sync with "master" video.
    """
    #반복재생하여 무한 반복 되도록 함.
    def __init__(self, data_queue, master=None):
        QtWidgets.QMainWindow.__init__(self, master)
        self.setWindowTitle("Mini Player")
        self.statusbar = self.statusBar()
        self.statusbar.showMessage("Ready")

        # Create a basic vlc instance
        self.instance = vlc.Instance('--input-repeat=9999')

        self.media = None

        # Create an empty vlc media player
        self.mediaplayer = self.instance.media_player_new()

        self.init_ui()
        self.open_file()
        print("---")
        self.mediaplayer.event_manager().event_attach(vlc.EventType.MediaPlayerEndReached, self.state_changed)
        self.timer = QtCore.QTimer(self)
        self.timer.setInterval(10)
        self.timer.timeout.connect(self.update_ui)

        self.data_queue = data_queue
        self.timer.start()

    def state_changed(self, event):
        if event.type == vlc.EventType.MediaPlayerEndReached:
            if self.mediaplayer.get_state() == vlc.State.Ended:
                print("영상 종료")
                self.instance.release()  # VLC 인스턴스 종료
                QtWidgets.QApplication.quit()  # PyQt5 응용 프로그램 종료
                

    
    def init_ui(self):
        """Set up the user interface
        """
        self.widget = QtWidgets.QWidget(self)
        self.setCentralWidget(self.widget)
        self.setWindowFlags(QtCore.Qt.WindowStaysOnTopHint)

        # In this widget, the video will be drawn
        if platform.system() == "Darwin":  # for MacOS
            self.videoframe = QtWidgets.QMacCocoaViewContainer(0)
        else:
            self.videoframe = QtWidgets.QFrame()

        self.palette = self.videoframe.palette()
        self.palette.setColor(QtGui.QPalette.Window, QtGui.QColor(0, 0, 0))
        self.videoframe.setPalette(self.palette)
        self.videoframe.setAutoFillBackground(True)

        self.vboxlayout = QtWidgets.QVBoxLayout()
        self.vboxlayout.addWidget(self.videoframe)
        self.widget.setLayout(self.vboxlayout)
        
        
        #테스트를 위한 10초 넘기기
        self.layout = QtWidgets.QHBoxLayout()
        self.vboxlayout.addLayout(self.layout)
        self.skip_btn = QtWidgets.QPushButton("[테스트용]10초뒤로")
        self.layout.addWidget(self.skip_btn)
        self.skip_btn.clicked.connect(self.skip)
        #테스트를 위한 10초 넘기기


        
    def skip(self): #테스트를 위한 10초 넘기기
        current = self.mediaplayer.get_time() #현재 재생 시간을 받아옴
        new = current + 10000 #밀리초 단위로 계산
        self.mediaplayer.set_time(new)

    #다운로드 된 영상의 재생을 위해 저장된 위치를 찾을 수 있도록 구현
    def open_file(self):
        """Open a media file in a MediaPlayer
        """
        filename = os.path.join(os.getcwd(), '저장 경로 템플릿')

        if not os.path.isfile(filename): #예외처리문
            QtWidgets.QMessageBox.critical(self, "Error", "파일을 못찾음")
            return

        self.media = self.instance.media_new(filename)

        # Put the media in the media player
        self.mediaplayer.set_media(self.media)

        # Parse the metadata of the file
        self.media.parse()

        # Set the title of the track as the window title
        self.setWindowTitle("오픈소스SW Team4 Digital Signage")

        # The media player has to be 'connected' to the QFrame (otherwise the
        # video would be displayed in it's own window). This is platform
        # specific, so we must give the ID of the QFrame (or similar object) to
        # vlc. Different platforms have different functions for this
        if platform.system() == "Linux":  # for Linux using the X Server
            self.mediaplayer.set_xwindow(int(self.videoframe.winId()))
        elif platform.system() == "Windows":  # for Windows
            self.mediaplayer.set_hwnd(int(self.videoframe.winId()))
        elif platform.system() == "Darwin":  # for MacOS
            self.mediaplayer.set_nsobject(int(self.videoframe.winId()))

        # Start playing the video as soon as it loads
        self.mediaplayer.play()

    def update_ui(self):
        self.update_statusbar()

        try:
            val = self.data_queue.get(block=False)
        except queue.Empty:
            return

        if val == '<':
            self.mediaplayer.set_rate(self.mediaplayer.get_rate() * 0.5)
            return
        if val == '>':
            self.mediaplayer.set_rate(self.mediaplayer.get_rate() * 2)
            return
        if val == 'P':
            self.mediaplayer.play()
            return
        if val == 'p':
            self.mediaplayer.pause()
            return
        if val == 'S':
            self.mediaplayer.stop()
            return

        val = int(val)
        if val != self.mediaplayer.get_time():
            self.mediaplayer.set_time(val)

    def update_statusbar(self):
        mtime = QtCore.QTime(0, 0, 0, 0)
        time = mtime.addMSecs(self.mediaplayer.get_time())
        self.statusbar.showMessage(time.toString())
    #속도 바꿔보기    
    def set_rate(self, rate):
        return self.mediaplayer.set_rate(rate)

# 제공된 유튜브 다운로드 코드를 추가하여 실행 시 지정된 링크의 유튜브 영상을 다운받을 수 있게 구현
def my_hook(d):
    if d['status'] == 'finished':
        print('Done downloading, now converting ...')

ydl_opts = {
    'download_archive': 'archive.txt',
    'ignoreerrors': True,
    'nooverwrites': True,
    'format': 'bestvideo[ext=mkv]+bestaudio/best[ext=mkv]/best',  #현재 ext=mkv시 병합이 오류없이 이루어짐(기존height<=1080에서 ext=mkv로 변경)
    'outtmpl': '저장 경로 템플릿',        
    'noplaylist' : False,       
    'progress_hooks': [my_hook],  
}

    
def main():
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download(['https://www.youtube.com/watch?v=Lr9xbHtpAfU']) #다운받을 유튜브 영상 주소
        
    app = QtWidgets.QApplication(sys.argv)

    data_queue = queue.Queue()

    player = MiniPlayer(data_queue)
    player.show()
    player.resize(1280, 780)
    player.set_rate(2) #2배속 
    sys.exit(app.exec_())
    


    
if __name__ == "__main__":
    main()

OSError: dlopen(libvlccore.dylib, 0x0006): tried: 'libvlccore.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibvlccore.dylib' (no such file), '/Users/kim-sung-eun/anaconda3/lib/python3.10/lib-dynload/../../libvlccore.dylib' (no such file), '/Users/kim-sung-eun/anaconda3/bin/../lib/libvlccore.dylib' (no such file), '/usr/lib/libvlccore.dylib' (no such file, not in dyld cache), 'libvlccore.dylib' (no such file), '/usr/local/lib/libvlccore.dylib' (no such file), '/usr/lib/libvlccore.dylib' (no such file, not in dyld cache)